In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [81]:
import time

In [2]:
url = 'https://www.fca.org.uk/news/search-results?category=warnings&start=1'
response = requests.get(url, headers={"Accept-Language":"en-US"})
soup = BeautifulSoup(response.content, "html.parser")

In [77]:
#This is the total number of company names listed on the FCA watch-list 

search_results = int(soup.find('p', class_='search-results__count col-xs-12').find('strong').find_next('strong').find_next('strong').string)

search_results


10351

In [102]:
#Get FCA URLs, dates and names for companies flagged  

def get_fca_comp_info():
    df = {}
    comp_names=[]
    websites = []
    dates =[]   
    for page in range(1, search_results, 10):
        print(f'scraping page {page}')
        response = requests.get(f'https://www.fca.org.uk/news/search-results?category=warnings&start={page}', headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
        try:
            for company in soup.find_all('li', class_='search-item'):
                fca_urls = company.find('a')
                websites.append(fca_urls['href'])
                fca_dates = company.find('span').find_next('span').text
                dates.append(fca_dates.split()[1])
                names = company.find('a').text.strip()
                comp_names.append(names)
                df={'comp_name':comp_names, 'website': websites, 'date': dates}
                output_df = pd.DataFrame.from_dict(df)#, orient='index')
                output_df.to_csv('fca-comp-info-scaped.csv')
        except:
            print(f'error on page {page}')
        time.sleep(3)
    return output_df

In [103]:
#Call function to get output_df 
get_fca_comp_info()

scraping page 1
scraping page 11
scraping page 21
scraping page 31
scraping page 41


,comp_name,website,date
0,SMARTEXFX,https://www.fca.org.uk/news/warnings/smartexfx,02/12/2022
1,BelleoFX,https://www.fca.org.uk/news/warnings/belleofx,02/12/2022
2,Satelys Ltd,https://www.fca.org.uk/news/warnings/satelys-ltd,02/12/2022
3,Auto FX Trade,https://www.fca.org.uk/news/warnings/auto-fx-t...,02/12/2022
4,cryptocreek-mining.com,https://www.fca.org.uk/news/warnings/cryptocre...,02/12/2022
5,ULTIMATETRUSTFX.UK,https://www.fca.org.uk/news/warnings/ultimatet...,01/12/2022
6,FX Trading Station,https://www.fca.org.uk/news/warnings/fx-tradin...,01/12/2022
7,COINTRUSTFX,https://www.fca.org.uk/news/warnings/cointrustfx,01/12/2022
8,Debt Help,https://www.fca.org.uk/news/warnings/debt-help,01/12/2022
9,STOCK BIT FX,https://www.fca.org.uk/news/warnings/stock-bit-fx,01/12/2022


In [104]:
#Read the csv you have downloaded (adjust the path)
df = pd.read_csv('fca-comp-info-scaped_v1.csv')
df.head()

,Unnamed: 0,comp_name,website,date
0,0,SMARTEXFX,https://www.fca.org.uk/news/warnings/smartexfx,02/12/2022
1,1,BelleoFX,https://www.fca.org.uk/news/warnings/belleofx,02/12/2022
2,2,Satelys Ltd,https://www.fca.org.uk/news/warnings/satelys-ltd,02/12/2022
3,3,Auto FX Trade,https://www.fca.org.uk/news/warnings/auto-fx-t...,02/12/2022
4,4,cryptocreek-mining.com,https://www.fca.org.uk/news/warnings/cryptocre...,02/12/2022


In [124]:
#Get URLs of companies flagged by FCA 

def get_fca_urls():
    urls = []
    for fca_url in list(df['website']):
        response = BeautifulSoup(requests.get(fca_url).content, "html.parser")
    
        try:
            for tag in response.find_all('p'):
            text = tag.text
            website_tag = "Website: "
            if text.startswith(website_tag):
                website = text[len(website_tag):]
                urls.append([website])
                df_URLs=pd.DataFrame(np.array(urls))
                #You can add for csv to save for every 500 pages 
                df_URLs.to_csv('fca-URLs.csv')
        except: 
            print(f'error in page {fca_url}')
        time.sleep(3)
            
    return df_URLs

In [125]:
#Call function to get all the URLs 
get_fca_urls()

,0
0,www.smartexfx.com
1,https://belleofx.com/
2,https://satelysfx.com/
3,https://autofxtrade.net/
4,https://cryptocreek-mining.com/
5,http://ultimatetrustfx.uk/
6,http://www.fxtradingstation.co.uk/
7,www.cointrustfx.online
8,www.debt-help.org
9,www.stocksbitfxltd.com


In [ ]:
#Standardise URLs (add 'http' where needed)

In [ ]:
#Check and keep only active ones 